In [136]:
import data_loader
import numpy as np
import pandas

In [214]:
# load data and assign names
trdf, valdf = data_loader.load_train_data("data/adult.data", is_df=True)
## adding columns labels https://chartio.com/resources/tutorials/how-to-rename-columns-in-the-pandas-python-library/
trdf.columns = ["age","workclass","fnlwgt","education","education-num","marital-status","occupation","relationship","race","sex","capital-gain","capital-loss","hours-per-week","native-country"
,"target"]
valdf.columns = ["age","workclass","fnlwgt","education","education-num","marital-status","occupation","relationship","race","sex","capital-gain","capital-loss","hours-per-week","native-country"
,"target"]

**Assign correct data type to categorical values**

In [224]:
catcols = [1,3,5,6,7,8,9,13,14]
trdf.iloc[:,catcols] = trdf.iloc[:,catcols].astype('category')
## Give category codes to each value in every feature
trdf.iloc[:,catcols] = trdf.iloc[:,catcols].apply(lambda x: x.cat.codes)

**Covariance of dataFrame**

In [227]:
trdf.cov()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
age,185.891905,-0.003940,-1.138767e+05,-0.671909,1.234029,-5.334731,-1.479055,-5.733177,0.343841,0.558840,7.862484e+03,316.597658,11.735440,-0.377665,1.369070
workclass,-0.003940,2.127632,-2.770643e+03,0.126433,0.194784,-0.142736,1.584990,-0.208654,0.058294,0.067421,3.781037e+02,7.642302,2.523059,-0.055843,0.030969
fnlwgt,-113876.698723,-2770.643229,1.117520e+10,-11403.024056,-12092.624657,4065.911048,868.117052,2134.231328,-2013.477669,1258.793770,-2.394274e+05,-457207.186435,-25547.374235,-44081.990082,-566.083498
education,-0.671909,0.126433,-1.140302e+04,14.966273,3.594278,-0.222731,-0.377533,-0.072514,0.044678,-0.050405,8.379416e+02,22.817745,2.624525,2.099443,0.130619
education-num,1.234029,0.194784,-1.209262e+04,3.594278,6.640001,-0.278588,1.186460,-0.395095,0.070870,0.018535,2.329426e+03,85.909957,4.802482,1.172877,0.374865
marital-status,-5.334731,-0.142736,4.065911e+03,-0.222731,-0.278588,2.277466,-0.069482,0.453634,-0.090562,-0.092987,-4.975877e+02,-21.135970,-3.590742,-0.264091,-0.129636
occupation,-1.479055,1.584990,8.681171e+02,-0.377533,1.186460,-0.069482,17.851317,-0.495868,0.025180,0.163098,7.616599e+02,31.172913,4.086689,-0.341067,0.131173
relationship,-5.733177,-0.208654,2.134231e+03,-0.072514,-0.395095,0.453634,-0.495868,2.585471,-0.157089,-0.441642,-6.907166e+02,-39.892856,-4.967651,-0.023882,-0.173001
race,0.343841,0.058294,-2.013478e+03,0.044678,0.070870,-0.090562,0.025180,-0.157089,0.717753,0.034906,8.591011e+01,7.110555,0.435754,0.877855,0.026020
sex,0.558840,0.067421,1.258794e+03,-0.050405,0.018535,-0.092987,0.163098,-0.441642,0.034906,0.221536,1.696870e+02,8.737295,1.344845,-0.034154,0.043404


## Data Exploration

In [215]:
# missign values replaced as np.nan
trdf = trdf.replace(' ?',np.nan)
valdf = valdf.replace(' ?',np.nan)

## Covariance of features - Principal component analysis to remove redundant features
[covariance finding](https://towardsdatascience.com/handling-missing-values-in-machine-learning-part-2-222154b4b58e)

In [212]:
trdf.cov()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
age,185.891905,-1.138767e+05,1.234029,7.862484e+03,316.597658,11.735440
fnlwgt,-113876.698723,1.117520e+10,-12092.624657,-2.394274e+05,-457207.186435,-25547.374235
education-num,1.234029,-1.209262e+04,6.640001,2.329426e+03,85.909957,4.802482
capital-gain,7862.484332,-2.394274e+05,2329.426093,5.563168e+07,-96243.069604,7364.117655
capital-loss,316.597658,-4.572072e+05,85.909957,-9.624307e+04,163318.679508,262.562840
hours-per-week,11.735440,-2.554737e+04,4.802482,7.364118e+03,262.562840,151.585174


## Analyze features with missing values

### Missing values in categorical data

In [201]:
catcols = [1,3,5,6,7,8,9,13,14] # categorical cols
# trdf[catcols]
pandas.options.mode
trdf.isna()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
24337,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
17049,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
21016,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2790,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
13511,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
29288,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2161,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
12546,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
210,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
20261,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


### Missing values in continuous data
[Missing value imputation in scikit](https://towardsdatascience.com/how-to-handle-missing-data-8646b18db0d4)

[Missing values strategoes](https://towardsdatascience.com/handling-missing-values-in-machine-learning-part-2-222154b4b58e)

# Models

## Naive Bayes - Baseline model


[Selecting subsets of data](https://medium.com/dunder-data/selecting-subsets-of-data-in-pandas-6fcd0170be9c)

TODO: missing values

In [120]:
# df = data_loader.load_train_data("data/adult.data")
df = np.genfromtxt("data/adult.data",delimiter=",", dtype="|U", autostrip=True)
df.shape

(32561, 15)

In [122]:
catcols = [1,3,5,6,7,8,9,13,14]
trdf[catcols] = trdf[catcols].astype('category')
trdf[catcols] = trdf[catcols].apply(lambda x: x.cat.codes)
# trdf
# trdf[1].cat.codes
trdf[14].shape

(29315,)

In [123]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
learner = model.fit(trdf.iloc[:,0:14], trdf.iloc[:,14])


In [124]:

catcols = [1,3,5,6,7,8,9,13,14]
valdf[catcols] = valdf[catcols].astype('category')
valdf[catcols] = valdf[catcols].apply(lambda x: x.cat.codes)
# trdf
# trdf[1].cat.codes
trdf[14].shape

(29315,)

In [125]:
learner.predict(valdf.iloc[:,0:14])

array([0, 1, 1, ..., 0, 0, 1], dtype=int8)

In [126]:
learner.score(valdf.iloc[:,0:14], valdf.iloc[:,14])

0.8009858287122612

## kNN


### 1 nearest

### 5 nearest

## ID3
### Parameterization

## NbTree
### Parameterization

## Neural Network
### Parameterization

## Support vector machine
### Parameterization